In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from google_play_scraper import Sort, reviews, reviews_all, app
import requests, lxml, re, json
import os
from pymongo import MongoClient

In [2]:
app_infos = []

for ap in tqdm(["com.vidio.android"]):
    info = app(ap, lang="id", country="id")
    del info["comments"]
    app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


In [3]:
df_app_infos = pd.DataFrame(app_infos).T
df_app_infos.columns = df_app_infos.iloc[0]
df_app_infos = df_app_infos.iloc[1:]
df_app_infos

title,"Vidio: Sports, Movies, Series"
description,Platform streaming terlengkap di Indonesia. No...
descriptionHTML,Platform streaming terlengkap di Indonesia. No...
summary,"TV, Live Sports, Original Series, Film Hollywo..."
installs,50.000.000+
minInstalls,50000000
realInstalls,72387906
score,3.664386
ratings,627601
reviews,244996
histogram,"[166250, 17582, 30234, 60011, 353520]"


In [8]:
# result = reviews_all(
#     "com.vidio.android",
#     sleep_milliseconds=0,
#     lang="id",
#     country="id",
#     sort=Sort.NEWEST
# )

In [ ]:
df = pd.DataFrame(result)
df = df.fillna("empty")
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,8957e3d5-69aa-41d5-9f6d-f057ec1f07f2,Usman Kuman,https://play-lh.googleusercontent.com/a-/ACB-R...,Nonon film❎ Nonton iklan✅,1,0,6.0.5-f68a5157fa,2023-03-22 17:57:37,"Hai Sahabat Vidio, ada banyak tayangan yang bi...",2023-03-22 18:06:27
1,96162329-51a0-4791-b968-c2b15cfc80f2,echa maria,https://play-lh.googleusercontent.com/a-/ACB-R...,"Saya udah lama pake apk ini,tp gak tau kenapa ...",3,0,6.0.5-f68a5157fa,2023-03-22 17:56:15,"Hai Sahabat Vidio, mohon maaf atas ketidaknyam...",2023-03-22 18:06:07
2,21051938-10c1-41ff-bbdf-3913f8d3a624,Putri Cahyani,https://play-lh.googleusercontent.com/a/AGNmyx...,Terlalu banyak iklan,1,0,empty,2023-03-22 17:42:04,"Hai Sahabat Vidio, ada banyak tayangan yang bi...",2023-03-22 17:48:40
3,1b0eb4a4-1309-4e9a-a759-d251a44a0c29,Sulfiannur Ahmad,https://play-lh.googleusercontent.com/a-/ACB-R...,q🤣aaaaA,1,0,empty,2023-03-22 16:57:18,"Hai Kak Sulfiannur, perihal apa yang dapat kam...",2023-03-22 17:47:56
4,17c75440-9188-4ad9-b08e-6159dea21a23,angga fajar,https://play-lh.googleusercontent.com/a-/ACB-R...,saya bru,3,0,empty,2023-03-22 16:45:20,Hai Kak Angga. Terima kasih telah menggunakan ...,2023-03-22 17:27:07


In [7]:
client = MongoClient(
    os.environ["MONGODB_URL"],
    serverSelectionTimeoutMS=300000
)
db = client["vidio"]
collection = db["google_play_store_reviews"]
df_dict = df.to_dict("records")

batch_size = 1_000
num_records = len(df_dict)
num_batches = num_records // batch_size

if num_records % batch_size != 0:
    num_batches += 1

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, num_records)
    batch = df_dict[start_idx:end_idx]
    
    if batch:
        collection.insert_many(batch)